### Read config file

In [1]:
import numpy as np
import obspy
from obspy import UTCDateTime
from concurrent.futures import ThreadPoolExecutor
import os
import time
from geopy.distance import great_circle
import folium
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import yaml
import shutil

In [2]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import maplib
from toollib_standard import mathlib

In [3]:
with open('a-project.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project/output_FJSJ_16-01/'               # Harmon server
name_project

'project/output_regular_v9.5_all/'

In [4]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = dir_project_workspace + name_project
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project/output_regular_v9.5_all/


In [5]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

### parameter settings

In [6]:
tag = info_basic['tag']
stalistname_all = info_basic['stalistname_all']

### Generat sta_info

In [7]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = mathlib.GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist() 
lat_stations_all =  stainfo['latitude'].tolist() 
lon_stations_all =  stainfo['longitude'].tolist() 
elevation_stations_all = stainfo['elevation'].tolist()

In [8]:
sta_info = {}
years = {2014}
for i in range(nsta_all):
    sta_info[stalist_all[i]] = {}
    sta_info[stalist_all[i]]['year'] = 2014
    sta_info[stalist_all[i]]['lat'] =float(lat_stations_all[i])
    sta_info[stalist_all[i]]['lon'] =float(lon_stations_all[i])

### Generate subworks

In [10]:
stalist_all

['R0101',
 'R0102',
 'R0103',
 'R0104',
 'R0105',
 'R0106',
 'R0107',
 'R0108',
 'R0109',
 'R0110',
 'R0111',
 'R0112',
 'R0113',
 'R0114',
 'R0115',
 'R0116',
 'R0117',
 'R0118',
 'R0301',
 'R0302',
 'R0303',
 'R0304',
 'R0305',
 'R0306',
 'R0307',
 'R0308',
 'R0309',
 'R0310',
 'R0311',
 'R0312',
 'R0313',
 'R0314',
 'R0315',
 'R0316',
 'R0317',
 'R0318',
 'R0319',
 'R0320',
 'R0501',
 'R0502',
 'R0503',
 'R0504',
 'R0505',
 'R0506',
 'R0507',
 'R0508',
 'R0509',
 'R0510',
 'R0511',
 'R0512',
 'R0513',
 'R0514',
 'R0515',
 'R0516',
 'R0517',
 'R0518',
 'R0519',
 'R0520',
 'R0701',
 'R0702',
 'R0703',
 'R0704',
 'R0705',
 'R0706',
 'R0707',
 'R0708',
 'R0709',
 'R0710',
 'R0711',
 'R0712',
 'R0713',
 'R0714',
 'R0715',
 'R0716',
 'R0717',
 'R0718',
 'R0719',
 'R0720',
 'R0901',
 'R0902',
 'R0903',
 'R0904',
 'R0905',
 'R0906',
 'R0907',
 'R0908',
 'R0909',
 'R0910',
 'R0911',
 'R0912',
 'R0913',
 'R0914',
 'R0915',
 'R0916',
 'R0917',
 'R0918',
 'R0919',
 'R0920',
 'R1101',
 'R1102',


In [14]:
key_subworks = ['all']
info_basic['key_subworks'] = key_subworks
key_subworks

['all']

In [13]:
dir_partition = dir_project + 'partition/'
info_basic['rdir_partition'] = 'partition/'
if os.path.exists(dir_partition):
    shutil.rmtree(dir_partition)
os.makedirs(dir_partition)

In [15]:
for key_subwork in key_subworks:
    Station = stalist_all
    lat = [sta_info[key]['lat'] for key in stalist_all]
    lon = [sta_info[key]['lon'] for key in stalist_all]
    filename = dir_partition + key_subwork + '.txt'
    np.savetxt(filename, np.array([Station, lat, lon]).T, fmt='%s', delimiter=' ', header='Station lat lon')

In [16]:
#np.save(info_basic['dir_config']+'/Basic_info.npy', info_basic)
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)